In [1]:
from openai import OpenAI
from tool import Tool
from weatherer.weather_api import get_weather_today
from munch import DefaultMunch

import requests
import instructor
import json

API_URL = 'http://localhost:11434/api'


In [2]:
# enables `response_model` in create call
client = instructor.patch(
    OpenAI(
        base_url="http://localhost:11434/v1",
        api_key="ollama",
    ),
    mode=instructor.Mode.JSON,
)

tools = {
    'get_weather': Tool(
        definition={
            "type": "function",
            "function": {
                "name": "get_weather",
                "description": "Get today's weather from a location.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {"type": "string", "description": "The location to get the weather from."}
                    },
                    "required": ["location"]
                }
            }
        },
        function=get_weather_today
    )
}


def tool_defs(): return [x.definition for x in tools.values()]

In [3]:
def generate_tool_outputs(tool_calls):
    tool_outputs = []

    for tool_call in tool_calls:
        function_name = tool_call.function.name
        arguments = tool_call.function.arguments.toDict()

        if function_name in tools:
            output = tools[function_name].call(**arguments)
            print(f'Tool output ({function_name}{arguments}): {output}')
            tool_outputs.append({
                'role': 'system',
                'function_name': function_name,
                'content': str(output)
            })

    return tool_outputs


In [4]:
def run_assistant(message):
    chat_messages = [
        {
            "role": "user",
            "content": message,
        }
    ]

    res = client.chat.completions.create(
        model="weatherer",
        messages=chat_messages,
        tools=tool_defs(),
        tool_choice='auto',
        max_retries=3,
    )

    print(res.model_dump_json(indent=2))

    # Check if there is a tool call in the response
    tool_calls = res.choices[0].message.tool_calls
    if tool_calls:
        tool_outputs = generate_tool_outputs(tool_calls)
        chat_messages.extend(tool_outputs)

        for chunk in client.chat.completions.create(
            model="weatherer",
            messages=chat_messages,
            max_retries=3,
            stream=True
        ):
            print(chunk.choices[0].delta.content, end='')


## Mistral RAW MODE

In [5]:
def run_assistant_raw(message):
    chat_messages = [
        {
            "role": "user",
            "content": message,
        }
    ]

    res: str = json.loads(requests.post(f"{API_URL}/generate", json={
        "model": "weatherer",
        "prompt": f"[AVAILABLE_TOOLS] {json.dumps(tool_defs())} [/AVAILABLE_TOOLS][INST] {message} [/INST]",
        "raw": True,
        "stream": False
    }).content)["response"]

    print("Response:", res)

    # Check if there is a tool call in the response
    if "[TOOL_CALLS]" in res:
        try:
            called_tools = [json.loads(x)[0] for x in res.splitlines()[0].split("[TOOL_CALLS] ")[1].split('$$')]
            print("Called Tools:", called_tools)

            tool_calls = [DefaultMunch.fromDict({'function': t}) for t in called_tools]

            tool_outputs = generate_tool_outputs(tool_calls)
            chat_messages.extend(tool_outputs)

        except Exception as e:
            print("Error parsing tool calls:")
            raise e

        chat_messages.append({
            "role": "user",
            "content": f"With the available data, answer this prompt, indicating today's weather and hourly forecast: {message}. Do not mention your functions. Do not make up any data.",
        })
        
        print("Context Length:", len(str(chat_messages)))
        print("Final Response:\n")

        for chunk in client.chat.completions.create(
            model="weatherer",
            messages=chat_messages,
            max_retries=3,
            stream=True
        ):
            print(chunk.choices[0].delta.content, end='')


In [7]:
# run_assistant_raw("What's the weather in Madrid, Spain? Use Celsius")
run_assistant_raw("What's the weather in El Molar, Madrid? Use Celsius")

Response: [TOOL_CALLS] [{"name": "get_weather", "arguments": {"location": "El Molar, Madrid"}}]
Called Tools: [{'name': 'get_weather', 'arguments': {'location': 'El Molar, Madrid'}}]
Tool output (get_weather{'location': 'El Molar, Madrid'}): {"Today's weather forecast at El Molar, Madrid": {'location': {'name': 'El Molar', 'region': 'Madrid', 'country': 'Spain', 'lat': 40.73, 'lon': -3.58, 'localtime': '2024-07-24 23:49'}, 'current': {'last_updated': '2024-07-24 23:45', 'temp_c': 29.4, 'condition': 'Clear', 'wind_kph': 6.8, 'wind_dir': 'NNE', 'humidity': 20, 'cloud': 0, 'feelslike_c': 27.2, 'vis_km': 10.0, 'gust_kph': 14.4}, 'forecast': {'date': '2024-07-24', 'day': {'maxtemp_c': 37.3, 'mintemp_c': 24.0, 'avgtemp_c': 30.7, 'maxwind_kph': 12.6, 'totalsnow_cm': 0.0, 'avgvis_km': 10.0, 'avghumidity': 21, 'daily_chance_of_rain': 0, 'daily_chance_of_snow': 0, 'condition': 'Sunny'}, 'hour': [{'time': '2024-07-24 00:00', 'temp_c': 27.2, 'condition': 'Clear ', 'wind_kph': 8.3, 'wind_dir': 'NNW